In [ ]:
import requests
import json
import time
from datetime import datetime, timedelta
import csv
from tqdm import tqdm

BASE_URL = "http://opendata.trudvsem.ru/api/v1/vacancies/region"
LIMIT = 100
SLEEP = 0.2
RETRY_DELAY = 15

REGIONS = {
    "7700000000000": "Москва",
    "7800000000000": "Санкт-Петербург",
    "9200000000000": "Севастополь",
    "0100000000000": "Республика Адыгея",
    "0200000000000": "Республика Башкортостан",
    "0300000000000": "Республика Бурятия",
    "0400000000000": "Республика Алтай",
    "0500000000000": "Республика Дагестан",
    "0600000000000": "Республика Ингушетия",
    "0700000000000": "Кабардино-Балкарская Республика",
    "0800000000000": "Республика Калмыкия",
    "0900000000000": "Карачаево-Черкесская Республика",
    "1000000000000": "Республика Карелия",
    "1100000000000": "Республика Коми",
    "1200000000000": "Республика Марий Эл",
    "1300000000000": "Республика Мордовия",
    "1400000000000": "Республика Саха (Якутия)",
    "1500000000000": "Республика Северная Осетия - Алания",
    "1600000000000": "Республика Татарстан",
    "1700000000000": "Республика Тыва",
    "1800000000000": "Удмуртская Республика",
    "1900000000000": "Республика Хакасия",
    "2000000000000": "Чеченская Республика",
    "2100000000000": "Чувашская Республика",
    "2200000000000": "Алтайский край",
    "2300000000000": "Краснодарский край",
    "2400000000000": "Красноярский край",
    "2500000000000": "Приморский край",
    "2600000000000": "Ставропольский край",
    "2700000000000": "Хабаровский край",
    "2800000000000": "Амурская область",
    "2900000000000": "Архангельская область",
    "3000000000000": "Астраханская область",
    "3100000000000": "Белгородская область",
    "3200000000000": "Брянская область",
    "3300000000000": "Владимирская область",
    "3400000000000": "Волгоградская область",
    "3500000000000": "Вологодская область",
    "3600000000000": "Воронежская область",
    "3700000000000": "Ивановская область",
    "3800000000000": "Иркутская область",
    "3900000000000": "Калининградская область",
    "4000000000000": "Калужская область",
    "4100000000000": "Камчатский край",
    "4200000000000": "Кемеровская область - Кузбасс",
    "4300000000000": "Кировская область",
    "4400000000000": "Костромская область",
    "4500000000000": "Курганская область",
    "4600000000000": "Курская область",
    "4700000000000": "Ленинградская область",
    "4800000000000": "Липецкая область",
    "4900000000000": "Магаданская область",
    "5000000000000": "Московская область",
    "5100000000000": "Мурманская область",
    "5200000000000": "Нижегородская область",
    "5300000000000": "Новгородская область",
    "5400000000000": "Новосибирская область",
    "5500000000000": "Омская область",
    "5600000000000": "Оренбургская область",
    "5700000000000": "Орловская область",
    "5800000000000": "Пензенская область",
    "5900000000000": "Пермский край",
    "6000000000000": "Псковская область",
    "6100000000000": "Ростовская область",
    "6200000000000": "Рязанская область",
    "6300000000000": "Самарская область",
    "6400000000000": "Саратовская область",
    "6500000000000": "Сахалинская область",
    "6600000000000": "Свердловская область",
    "6700000000000": "Смоленская область",
    "6800000000000": "Тамбовская область",
    "6900000000000": "Тверская область",
    "7000000000000": "Томская область",
    "7100000000000": "Тульская область",
    "7200000000000": "Тюменская область",
    "7300000000000": "Ульяновская область",
    "7400000000000": "Челябинская область",
    "7500000000000": "Забайкальский край",
    "7600000000000": "Ярославская область",
    "7900000000000": "Еврейская автономная область",
    "8300000000000": "Ненецкий автономный округ",
    "8600000000000": "Ханты-Мансийский автономный округ - Югра",
    "8700000000000": "Чукотский автономный округ",
    "8900000000000": "Ямало-Ненецкий автономный округ",
    "9100000000000": "Республика Крым",
    "9300000000000": "Донецкая Народная Республика",
    "9400000000000": "Луганская Народная Республика",
    "9500000000000": "Запорожская область",
    "9600000000000": "Херсонская область"
}


def fetch_vacancies(region_code, date_from, date_to):
    """
    Получаем вакансии по региону за конкретный интервал времени с корректным offset.
    """
    all_vacancies = []
    seen_ids = set()
    offset = 0

    while True:
        params = {
            "offset": offset,
            "limit": LIMIT,
            "modifiedFrom": date_from,
            "modifiedTo": date_to
        }

        try:
            r = requests.get(f"{BASE_URL}/{region_code}", params=params, timeout=30)
            r.raise_for_status()
            data = r.json()
        except Exception as e:
            print(f"[ERROR] {e}, ждём {RETRY_DELAY} секунд...")
            time.sleep(RETRY_DELAY)
            continue

        vacancies = data.get("results", {}).get("vacancies", [])
        if not vacancies:
            break

        new_count = 0
        for v in vacancies:
            vac = v.get("vacancy")
            if vac and vac.get("id") not in seen_ids:
                seen_ids.add(vac.get("id"))
                all_vacancies.append(vac)
                new_count += 1

        print(f"Offset {offset}: получено {new_count} новых вакансий")

        if len(vacancies) < LIMIT:
            break

        offset += LIMIT
        time.sleep(SLEEP)

    return all_vacancies


def split_interval(region_code, start, end):
    """
    Рекурсивно делим интервал: день → часы → минуты, пока не достанем все.
    """
    iso_from = start.strftime("%Y-%m-%dT%H:%M:%SZ")
    iso_to = end.strftime("%Y-%m-%dT%H:%M:%SZ")

    vacancies = fetch_vacancies(region_code, iso_from, iso_to)

    # если в интервале слишком много вакансий, дробим дальше
    if len(vacancies) >= LIMIT and (end - start) > timedelta(minutes=1):
        mid = start + (end - start) / 2
        return split_interval(region_code, start, mid) + \
               split_interval(region_code, mid + timedelta(seconds=1), end)
    else:
        return vacancies


def daterange_days(start_date, end_date):
    """Генератор по дням"""
    current = start_date
    while current < end_date:
        next_day = current + timedelta(days=1) - timedelta(seconds=1)
        yield current, min(next_day, end_date)
        current = next_day + timedelta(seconds=1)


def main():
    all_vacancies = []
    start_date = datetime(2025, 1, 1)
    end_date = datetime.now()

    for region_code, region_name in REGIONS.items():
        print(f"\n=== Сбор вакансий для региона {region_name} ===")
        region_vacancies = []

        for date_from, date_to in tqdm(daterange_days(start_date, end_date), desc=f"{region_name}", leave=False):
            day_vacancies = split_interval(region_code, date_from, date_to)
            region_vacancies.extend(day_vacancies)

        print(f"[INFO] {region_name}: собрано {len(region_vacancies)} вакансий")
        all_vacancies.extend(region_vacancies)

        # Промежуточное сохранение
        with open("all_vacancies_partial.json", "w", encoding="utf-8") as f:
            json.dump(all_vacancies, f, ensure_ascii=False, indent=2)

    # Финальное сохранение
    with open("all_vacancies.json", "w", encoding="utf-8") as f:
        json.dump(all_vacancies, f, ensure_ascii=False, indent=2)

    # CSV
    if all_vacancies:
        keys = set()
        for vac in all_vacancies:
            keys.update(vac.keys())
        keys = list(keys)

        with open("all_vacancies.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(all_vacancies)

    print(f"\nИТОГО собрано вакансий: {len(all_vacancies)}")
    print("Данные сохранены в all_vacancies.json и all_vacancies.csv")


if __name__ == "__main__":
    main()
